<a href="https://colab.research.google.com/github/SebaAyala/AST_TP_2/blob/main/AST_TP_2_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación de librerias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from xgboost import plot_importance, plot_tree
color_pal = sns.color_palette()
plt.style.use('tableau-colorblind10')

from google.colab import drive
from google.colab import auth
from google.auth import default

In [3]:
creds, _ = default()
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()

Mounted at /content/drive


## Carga de datasets

In [9]:
# Ruta al archivo en tu Google Drive
df_siniestros = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AST - Series Temporales/TP_2/siniestros.csv', encoding='latin1', sep=';')
df_precipitaciones = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AST - Series Temporales/TP_2/precip.xlsx', skiprows=3)
df_temperatura = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AST - Series Temporales/TP_2/tempBA.xlsx', skiprows=2, date_parser=['Fecha'])

/tmp/ipython-input-633792332.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_siniestros = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AST - Series Temporales/TP_2/siniestros.csv', encoding='latin1', sep=';')
/tmp/ipython-input-633792332.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_temperatura = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AST - Series Temporales/TP_2/tempBA.xlsx', skiprows=2, date_parser=['Fecha'])


In [10]:
print(df_siniestros.head())
print(df_precipitaciones.head())
print(df_temperatura.head())

          id_hecho  n_victimas       fecha  aaaa  mm  dd      hora  hh  \
0  LC-2019-0008283           2  2019-01-04  2019   1   4  17:53:00  17   
1  LC-2019-0007634           1  2019-01-04  2019   1   4  14:50:00  14   
2  LC-2019-0008974           1  2019-01-05  2019   1   5  09:30:00   9   
3  LC-2019-0010983           2  2019-01-06  2019   1   6  22:20:00  22   
4  LC-2019-0011092           1  2019-01-07  2019   1   7  03:50:00   3   

  direccion_normalizada calle  ... comuna tipo_de_calle geocodificacion_caba  \
0                    SD   NaN  ...     SD            SD                   SD   
1                    SD   NaN  ...     SD            SD                   SD   
2                    SD   NaN  ...     SD            SD                   SD   
3                    SD   NaN  ...     SD            SD                   SD   
4                    SD   NaN  ...     SD            SD                   SD   

  longitud latitud participantes    victima contraparte gravedad tipo_de_d